In [1]:
import os
from PIL import Image, ImageOps
import numpy as np
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

In [2]:
! curl -O https://challenge.blob.core.windows.net/challengefiles/gear_images.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33.4M  100 33.4M    0     0  5545k      0  0:00:06  0:00:06 --:--:-- 6272k


In [3]:
! tar xopf gear_images.zip

In [4]:
def normalize(arr):
    """
    Linear normalization
    http://en.wikipedia.org/wiki/Normalization_%28image_processing%29
    """
    arr = arr.astype('float')
    # Do not touch the alpha channel
    for i in range(3):
        minval = arr[...,i].min()
        maxval = arr[...,i].max()
        if minval != maxval:
            arr[...,i] -= minval
            arr[...,i] *= (255/(maxval-minval)) 
            arr[...,i] += 128
    return arr

In [5]:
rawImages = [] 
labels = []
labelMap = {}
count = 0
size = (128, 128)

def list_files(directory,prefix = ''):
    global count
    for filename in os.listdir(directory):
        if  filename.endswith(".jpeg") or filename.endswith(".jpg") or filename.endswith(".png"): 
            filePath = directory + "/" + filename
            img = Image.open(filePath)
            img.thumbnail(size)
            imgResized = Image.new('RGB', size, (0, 0, 0))
            imgWidth, imageHeight = img.size
            imgResized.paste(img, ((size[0]-imgWidth)//2,(size[1]-imageHeight)//2))
            imgArray = normalize(np.array(imgResized))
            #np.save(filePath + ".npy", imgArray)
            label = filePath.split(os.path.sep)[-2]
            if label not in labelMap.keys():
                labelMap[label] = count
                label = count
                count += 1
            else:
                label = labelMap[label]             
            rawImages.append(imgArray.flatten()) 
            labels.append([label])
                
                

In [6]:
path = "./gear_images/" 

dir_list = os.listdir(path)
for p in dir_list:
    sub_path = os.path.join(path , p)
    list_files(sub_path)

UnboundLocalError: local variable 'count' referenced before assignment

In [ ]:
rawImages = np.array(rawImages) 
labels = np.array(labels)
num_classes = len(labelMap)
print("[INFO] label mapping:",labelMap,num_classes)
print("[INFO] pixels matrix: {:.2f}MB".format(
    rawImages.nbytes / (1024 * 1000.0))) 